In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import classification_report
import joblib
import re

In [2]:
df = pd.read_csv('Resources/super_clean_job.csv')

In [3]:
df.dtypes

Job Title             object
Salary Estimate       object
Rating               float64
Company Name          object
Location              object
Headquarters          object
Size                  object
Type of ownership     object
Industry              object
Sector                object
Revenue               object
dtype: object

In [4]:
df.keys()

Index(['Job Title', 'Salary Estimate', 'Rating', 'Company Name', 'Location',
       'Headquarters', 'Size', 'Type of ownership', 'Industry', 'Sector',
       'Revenue'],
      dtype='object')

In [5]:
df.head(2)

,Job Title,Salary Estimate,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD)
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD)


In [6]:
# A quick look at 'Salary Estimate' column. Please note it has 88 unique salary ranges overlapping each other.
# This will need to be cleaned up by remapping with a new set of salary ranges.
print('Looking at Salary Estimate ...')
print(df['Salary Estimate'].unique())
print(df['Salary Estimate'].nunique())

Looking at Salary Estimate ...
['$37K-$66K (Glassdoor est.)' '$46K-$87K (Glassdoor est.)'
 '$51K-$88K (Glassdoor est.)' nan '$51K-$87K (Glassdoor est.)'
 '$59K-$85K (Glassdoor est.)' '$43K-$76K (Glassdoor est.)'
 '$60K-$110K (Glassdoor est.)' '$41K-$78K (Glassdoor est.)'
 '$45K-$88K (Glassdoor est.)' '$73K-$127K (Glassdoor est.)'
 '$84K-$90K (Glassdoor est.)' '$27K-$52K (Glassdoor est.)'
 '$42K-$74K (Glassdoor est.)' '$77K-$132K (Glassdoor est.)'
 '$98K-$114K (Glassdoor est.)' '$48K-$96K (Glassdoor est.)'
 '$26K-$47K (Glassdoor est.)' '$31K-$59K (Glassdoor est.)'
 '$47K-$81K (Glassdoor est.)' '$43K-$69K (Glassdoor est.)'
 '$49K-$112K (Glassdoor est.)' '$30K-$54K (Glassdoor est.)'
 '$55K-$103K (Glassdoor est.)' '$37K-$70K (Glassdoor est.)'
 '$57K-$103K (Glassdoor est.)' '$35K-$45K (Glassdoor est.)'
 '$42K-$66K (Glassdoor est.)' '$65K-$81K (Glassdoor est.)'
 '$113K-$132K (Glassdoor est.)' '$60K-$66K (Glassdoor est.)'
 '$73K-$82K (Glassdoor est.)' '$67K-$92K (Glassdoor est.)'
 '$42K-$76K 

In [7]:
# Create a Python dictionary of lists
alternateSalaryRanges = {
        'Salary Estimate': ['$24K-$48K (Glassdoor est.)','$26K-$47K (Glassdoor est.)', 
'$27K-$52K (Glassdoor est.)','$28K-$52K (Glassdoor est.)','$29K-$38K (Glassdoor est.)','$30K-$53K (Glassdoor est.)',
'$30K-$54K (Glassdoor est.)','$31K-$59K (Glassdoor est.)','$32K-$56K (Glassdoor est.)','$34K-$61K (Glassdoor est.)', 
'$35K-$42K (Glassdoor est.)','$35K-$45K (Glassdoor est.)','$35K-$67K (Glassdoor est.)','$36K-$67K (Glassdoor est.)', 
'$37K-$44K (Glassdoor est.)','$37K-$66K (Glassdoor est.)','$37K-$68K (Glassdoor est.)','$37K-$70K (Glassdoor est.)',
'$38K-$68K (Glassdoor est.)','$40K-$72K (Glassdoor est.)','$40K-$74K (Glassdoor est.)','$41K-$78K (Glassdoor est.)', 
'$41K-$86K (Glassdoor est.)','$42K-$66K (Glassdoor est.)','$42K-$74K (Glassdoor est.)','$42K-$76K (Glassdoor est.)', 
'$43K-$69K (Glassdoor est.)','$43K-$76K (Glassdoor est.)','$43K-$77K (Glassdoor est.)','$43K-$94K (Glassdoor est.)',
'$44K-$78K (Glassdoor est.)','$44K-$82K (Glassdoor est.)','$45K-$78K (Glassdoor est.)','$45K-$88K (Glassdoor est.)', 
'$46K-$102K (Glassdoor est.)','$46K-$86K (Glassdoor est.)','$46K-$87K (Glassdoor est.)','$47K-$74K (Glassdoor est.)', 
'$47K-$76K (Glassdoor est.)','$47K-$81K (Glassdoor est.)','$48K-$88K (Glassdoor est.)','$48K-$96K (Glassdoor est.)', 
'$49K-$112K (Glassdoor est.)','$49K-$91K (Glassdoor est.)','$50K-$86K (Glassdoor est.)','$50K-$93K (Glassdoor est.)', 
'$51K-$87K (Glassdoor est.)','$51K-$88K (Glassdoor est.)', '$51K-$93K (Glassdoor est.)','$53K-$104K (Glassdoor est.)', 
'$53K-$94K (Glassdoor est.)','$53K-$99K (Glassdoor est.)','$54K-$75K (Glassdoor est.)','$55K-$101K (Glassdoor est.)',
'$55K-$103K (Glassdoor est.)','$57K-$100K (Glassdoor est.)','$57K-$103K (Glassdoor est.)','$57K-$104K (Glassdoor est.)', 
'$57K-$67K (Glassdoor est.)','$57K-$70K (Glassdoor est.)','$58K-$93K (Glassdoor est.)','$59K-$85K (Glassdoor est.)', 
'$60K-$110K (Glassdoor est.)','$60K-$124K (Glassdoor est.)','$60K-$66K (Glassdoor est.)','$63K-$116K (Glassdoor est.)',
'$64K-$113K (Glassdoor est.)','$65K-$120K (Glassdoor est.)','$65K-$81K (Glassdoor est.)','$67K-$92K (Glassdoor est.)', 
'$68K-$87K (Glassdoor est.)','$69K-$127K (Glassdoor est.)','$73K-$127K (Glassdoor est.)','$73K-$82K (Glassdoor est.)', 
'$73K-$89K (Glassdoor est.)','$74K-$123K (Glassdoor est.)','$76K-$122K (Glassdoor est.)','$77K-$132K (Glassdoor est.)', 
'$78K-$104K (Glassdoor est.)','$82K-$116K (Glassdoor est.)','$84K-$90K (Glassdoor est.)','$89K-$151K (Glassdoor est.)', 
'$93K-$159K (Glassdoor est.)','$97K-$129K (Glassdoor est.)','$98K-$114K (Glassdoor est.)','$99K-$178K (Glassdoor est.)', 
'$110K-$190K (Glassdoor est.)','$113K-$132K (Glassdoor est.)'],
        'New Salary Estimate': ['$20K-$50K', '$20K-$50K','$20K-$50K', '$20K-$50K','$20K-$50K', '$20K-$50K','$20K-$50K',
'$20K-$50K','$20K-$50K', '$20K-$50K','$20K-$50K','$20K-$50K','$20K-$50K', '$20K-$50K','$20K-$50K', '$20K-$50K',
'$20K-$50K','$20K-$50K','$20K-$50K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K',
'$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K',
'$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$56K-$70K','$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K',
'$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K','$71K-$80K',
'$71K-$80K','$81K-$90K','$81K-$90K','$81K-$90K','$81K-$90K','$81K-$90K','$81K-$90K','$81K-$90K','$81K-$90K','$81K-$90K',
'$81K-$90K','$91K-$109K','$91K-$109K','$91K-$109K','$91K-$109K','$91K-$109K','$91K-$109K','$91K-$109K','$91K-$109K',
'$91K-$109K','$91K-$109K','$91K-$109K','$91K-$109K','$91K-$109K','$91K-$109K','$110K-$190K','$110K-$190K','$110K-$190K',
'$110K-$190K','$110K-$190K','$110K-$190K','$110K-$190K']  
}


In [8]:
# Create a Python dictionary
salarymap = {
'$24K-$48K (Glassdoor est.)':'$20K-$50K', 
'$26K-$47K (Glassdoor est.)':'$20K-$50K', 
'$27K-$52K (Glassdoor est.)':'$20K-$50K',
'$28K-$52K (Glassdoor est.)':'$20K-$50K',
'$29K-$38K (Glassdoor est.)':'$20K-$50K',
'$30K-$53K (Glassdoor est.)':'$20K-$50K',
'$30K-$54K (Glassdoor est.)':'$20K-$50K',
'$31K-$59K (Glassdoor est.)':'$20K-$50K',
'$32K-$56K (Glassdoor est.)':'$20K-$50K',
'$34K-$61K (Glassdoor est.)':'$20K-$50K', 
'$35K-$42K (Glassdoor est.)':'$20K-$50K',
'$35K-$45K (Glassdoor est.)':'$20K-$50K',
'$35K-$67K (Glassdoor est.)':'$20K-$50K',
'$36K-$67K (Glassdoor est.)':'$20K-$50K', 
'$37K-$44K (Glassdoor est.)':'$20K-$50K',
'$37K-$66K (Glassdoor est.)':'$20K-$50K',
'$37K-$68K (Glassdoor est.)':'$20K-$50K',
'$37K-$70K (Glassdoor est.)':'$20K-$50K',
'$38K-$68K (Glassdoor est.)':'$20K-$50K',
'$40K-$72K (Glassdoor est.)':'$56K-$70K',
'$40K-$74K (Glassdoor est.)':'$56K-$70K',
'$41K-$78K (Glassdoor est.)':'$56K-$70K',
'$41K-$86K (Glassdoor est.)':'$56K-$70K',
'$42K-$66K (Glassdoor est.)':'$56K-$70K',
'$42K-$74K (Glassdoor est.)':'$56K-$70K',
'$42K-$76K (Glassdoor est.)':'$56K-$70K', 
'$43K-$69K (Glassdoor est.)':'$56K-$70K',
'$43K-$76K (Glassdoor est.)':'$56K-$70K',
'$43K-$77K (Glassdoor est.)':'$56K-$70K',
'$43K-$94K (Glassdoor est.)':'$56K-$70K',
'$44K-$78K (Glassdoor est.)':'$56K-$70K',
'$44K-$82K (Glassdoor est.)':'$56K-$70K',
'$45K-$78K (Glassdoor est.)':'$56K-$70K',
'$45K-$88K (Glassdoor est.)':'$56K-$70K', 
'$46K-$102K (Glassdoor est.)':'$56K-$70K',
'$46K-$86K (Glassdoor est.)':'$56K-$70K',
'$46K-$87K (Glassdoor est.)':'$56K-$70K',
'$47K-$74K (Glassdoor est.)':'$56K-$70K', 
'$47K-$76K (Glassdoor est.)':'$56K-$70K',
'$47K-$81K (Glassdoor est.)':'$56K-$70K',
'$48K-$88K (Glassdoor est.)':'$56K-$70K',
'$48K-$96K (Glassdoor est.)':'$71K-$80K', 
'$49K-$112K (Glassdoor est.)':'$71K-$80K',
'$49K-$91K (Glassdoor est.)':'$71K-$80K',
'$50K-$86K (Glassdoor est.)':'$71K-$80K',
'$50K-$93K (Glassdoor est.)':'$71K-$80K', 
'$51K-$87K (Glassdoor est.)':'$71K-$80K',
'$51K-$88K (Glassdoor est.)':'$71K-$80K',
'$51K-$93K (Glassdoor est.)':'$71K-$80K',
'$53K-$104K (Glassdoor est.)':'$71K-$80K', 
'$53K-$94K (Glassdoor est.)':'$71K-$80K',
'$53K-$99K (Glassdoor est.)':'$71K-$80K',
'$54K-$75K (Glassdoor est.)':'$71K-$80K',
'$55K-$101K (Glassdoor est.)':'$71K-$80K',
'$55K-$103K (Glassdoor est.)':'$71K-$80K',
'$57K-$100K (Glassdoor est.)':'$71K-$80K',
'$57K-$103K (Glassdoor est.)':'$71K-$80K',
'$57K-$104K (Glassdoor est.)':'$81K-$90K',
'$57K-$67K (Glassdoor est.)':'$81K-$90K',
'$57K-$70K (Glassdoor est.)':'$81K-$90K',
'$58K-$93K (Glassdoor est.)':'$81K-$90K',
'$59K-$85K (Glassdoor est.)':'$81K-$90K', 
'$60K-$110K (Glassdoor est.)':'$81K-$90K',
'$60K-$124K (Glassdoor est.)':'$81K-$90K',
'$60K-$66K (Glassdoor est.)':'$81K-$90K',
'$63K-$116K (Glassdoor est.)':'$81K-$90K',
'$64K-$113K (Glassdoor est.)':'$81K-$90K',
'$65K-$120K (Glassdoor est.)':'$91K-$109K',
'$65K-$81K (Glassdoor est.)':'$91K-$109K',
'$67K-$92K (Glassdoor est.)':'$91K-$109K', 
'$68K-$87K (Glassdoor est.)':'$91K-$109K',
'$69K-$127K (Glassdoor est.)':'$91K-$109K',
'$73K-$127K (Glassdoor est.)':'$91K-$109K',
'$73K-$82K (Glassdoor est.)':'$91K-$109K', 
'$73K-$89K (Glassdoor est.)':'$91K-$109K',
'$74K-$123K (Glassdoor est.)':'$91K-$109K',
'$76K-$122K (Glassdoor est.)':'$91K-$109K',
'$77K-$132K (Glassdoor est.)':'$91K-$109K', 
'$78K-$104K (Glassdoor est.)':'$91K-$109K',
'$82K-$116K (Glassdoor est.)':'$91K-$109K',
'$84K-$90K (Glassdoor est.)':'$91K-$109K',
'$89K-$151K (Glassdoor est.)':'$110K-$190K', 
'$93K-$159K (Glassdoor est.)':'$110K-$190K',
'$97K-$129K (Glassdoor est.)':'$110K-$190K',
'$98K-$114K (Glassdoor est.)':'$110K-$190K',
'$99K-$178K (Glassdoor est.)':'$110K-$190K', 
'$110K-$190K (Glassdoor est.)':'$110K-$190K',
'$113K-$132K (Glassdoor est.)':'$110K-$190K'
}

In [9]:
# Find null values
for column in df.columns:
    print(f"Column {column} has {df[column].isnull().sum()} null values")

Column Job Title has 2 null values
Column Salary Estimate has 2 null values
Column Rating has 2 null values
Column Company Name has 2 null values
Column Location has 2 null values
Column Headquarters has 2 null values
Column Size has 2 null values
Column Type of ownership has 2 null values
Column Industry has 2 null values
Column Sector has 2 null values
Column Revenue has 2 null values


In [10]:
# Drop the null values
df.dropna(inplace=True)

In [11]:
# Create a new column for holding the new salary estimate

df = df.assign(NewSalaryEstimate="")
df.head(4)

,Job Title,Salary Estimate,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,NewSalaryEstimate
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),
2,"Senior Data Analyst, Insights & Analytics Team...",$37K-$66K (Glassdoor est.),3.4,Squarespace\n3.4,"New York, NY","New York, NY",1001 to 5000 employees,Company - Private,Internet,Information Technology,Unknown / Non-Applicable,
3,Data Analyst,$37K-$66K (Glassdoor est.),4.1,Celerity\n4.1,"New York, NY","McLean, VA",201 to 500 employees,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD),


In [12]:
df.head(2)

,Job Title,Salary Estimate,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,NewSalaryEstimate
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),


In [13]:
# Load up new column "NewSalaryEstimate" using lambda function with the new mapped salary ranges

df.loc[:, "NewSalaryEstimate"] = df.apply(
    lambda z: salarymap.get(z["Salary Estimate"], z["NewSalaryEstimate"]), axis=1
)

In [14]:
df.head(2)

,Job Title,Salary Estimate,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,Sector,Revenue,NewSalaryEstimate
0,"Data Analyst, Center on Immigration and Justic...",$37K-$66K (Glassdoor est.),3.2,Vera Institute of Justice\n3.2,"New York, NY","New York, NY",201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD),$20K-$50K
1,Quality Data Analyst,$37K-$66K (Glassdoor est.),3.8,Visiting Nurse Service of New York\n3.8,"New York, NY","New York, NY",10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD),$20K-$50K


In [15]:
# Let's look at the distribution of the new salary ranges
checker_counts = df['NewSalaryEstimate'].value_counts()
checker_counts

$56K-$70K      454
$20K-$50K      387
$71K-$80K      356
$91K-$109K     263
$81K-$90K      227
$110K-$190K    167
Name: NewSalaryEstimate, dtype: int64

In [16]:
print('Looking at New Salary Estimate ...')
print(df['NewSalaryEstimate'].unique())
print(df['NewSalaryEstimate'].nunique())

Looking at New Salary Estimate ...
['$20K-$50K' '$56K-$70K' '$71K-$80K' '$81K-$90K' '$91K-$109K'
 '$110K-$190K']
6


In [17]:
# No longer need to strip out the non-numeric character in the salary range estimate
# Strip out the " (Glassdoor est.)" from the end of the "Salary Estimate" column and the leading space before it.
# Next strip out the 'K' and the '$' from the 'Salary Estimate'
#df['Salary Estimate'] = df['Salary Estimate'].str.replace(' \(.*\)', '')
#df['Salary Estimate'] = df['Salary Estimate'].str.replace('K', '000')
#df['Salary Estimate'] = df['Salary Estimate'].str.replace('$', '')
#df['Salary Estimate'] = df['Salary Estimate'].str.replace('-', ' ')
#df['Salary Estimate'] = df['Salary Estimate'].astype('int')
#zz = df['Salary Estimate'][0].split()
#df['Salary Estimate']

In [18]:
X = df[['Rating','Location','Size','Type of ownership','Industry','Sector','Revenue']]
X

,Rating,Location,Size,Type of ownership,Industry,Sector,Revenue
0,3.2,"New York, NY",201 to 500 employees,Nonprofit Organization,Social Assistance,Non-Profit,$100 to $500 million (USD)
1,3.8,"New York, NY",10000+ employees,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$2 to $5 billion (USD)
2,3.4,"New York, NY",1001 to 5000 employees,Company - Private,Internet,Information Technology,Unknown / Non-Applicable
3,4.1,"New York, NY",201 to 500 employees,Subsidiary or Business Segment,IT Services,Information Technology,$50 to $100 million (USD)
4,3.9,"New York, NY",501 to 1000 employees,Company - Private,Sports & Recreation,"Arts, Entertainment & Recreation",$100 to $500 million (USD)
...,...,...,...,...,...,...,...
1851,4.1,"Broomfield, CO",51 to 200 employees,Company - Private,Computer Hardware & Software,Information Technology,$25 to $50 million (USD)
1852,2.5,"Denver, CO",51 to 200 employees,Company - Private,Staffing & Outsourcing,Business Services,Unknown / Non-Applicable
1853,2.9,"Centennial, CO",10000+ employees,Company - Public,Wholesale,Business Services,$10+ billion (USD)
1854,3.1,"Centennial, CO",201 to 500 employees,Company - Private,Enterprise Software & Network Solutions,Information Technology,$25 to $50 million (USD)


In [19]:
y = df['NewSalaryEstimate'].unique
y

<bound method Series.unique of 0        $20K-$50K
1        $20K-$50K
2        $20K-$50K
3        $20K-$50K
4        $20K-$50K
           ...    
1851    $91K-$109K
1852    $91K-$109K
1853    $91K-$109K
1854    $91K-$109K
1855    $91K-$109K
Name: NewSalaryEstimate, Length: 1854, dtype: object>

In [20]:
# One-hot encoding the entire dataframe
X_dummies = pd.get_dummies(X)
print(X_dummies.columns)
X_dummies

Index(['Rating', 'Location_Addison, TX', 'Location_Alachua, FL',
       'Location_Alameda, CA', 'Location_Alhambra, CA',
       'Location_Allegheny West, PA', 'Location_Allen, TX',
       'Location_American Fork, UT', 'Location_Anaheim, CA',
       'Location_Arcadia, CA',
       ...
       'Revenue_$10+ billion (USD)', 'Revenue_$100 to $500 million (USD)',
       'Revenue_$2 to $5 billion (USD)', 'Revenue_$25 to $50 million (USD)',
       'Revenue_$5 to $10 billion (USD)', 'Revenue_$5 to $10 million (USD)',
       'Revenue_$50 to $100 million (USD)',
       'Revenue_$500 million to $1 billion (USD)',
       'Revenue_Less than $1 million (USD)',
       'Revenue_Unknown / Non-Applicable'],
      dtype='object', length=379)


,Rating,"Location_Addison, TX","Location_Alachua, FL","Location_Alameda, CA","Location_Alhambra, CA","Location_Allegheny West, PA","Location_Allen, TX","Location_American Fork, UT","Location_Anaheim, CA","Location_Arcadia, CA",...,Revenue_$10+ billion (USD),Revenue_$100 to $500 million (USD),Revenue_$2 to $5 billion (USD),Revenue_$25 to $50 million (USD),Revenue_$5 to $10 billion (USD),Revenue_$5 to $10 million (USD),Revenue_$50 to $100 million (USD),Revenue_$500 million to $1 billion (USD),Revenue_Less than $1 million (USD),Revenue_Unknown / Non-Applicable
0,3.2,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,3.8,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,3.4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,4.1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,3.9,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851,4.1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1852,2.5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1853,2.9,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1854,3.1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [21]:
# Get the list of columns from the X_dummies dataframe
cols = list(X_dummies.columns.values)
cols

['Rating',
 'Location_Addison, TX',
 'Location_Alachua, FL',
 'Location_Alameda, CA',
 'Location_Alhambra, CA',
 'Location_Allegheny West, PA',
 'Location_Allen, TX',
 'Location_American Fork, UT',
 'Location_Anaheim, CA',
 'Location_Arcadia, CA',
 'Location_Arlington Heights, IL',
 'Location_Arlington, TX',
 'Location_Athens, GA',
 'Location_Aurora, CO',
 'Location_Austin, TX',
 'Location_Azusa, CA',
 'Location_Beech Grove, IN',
 'Location_Bellevue, WA',
 'Location_Bensalem, PA',
 'Location_Berkeley Heights, NJ',
 'Location_Berkeley, CA',
 'Location_Berwyn, PA',
 'Location_Beverly Hills, CA',
 'Location_Blue Bell, PA',
 'Location_Boothwyn, PA',
 'Location_Boulder, CO',
 'Location_Brea, CA',
 'Location_Bridgeview, IL',
 'Location_Broadview, IL',
 'Location_Bronx, NY',
 'Location_Brooklyn, NY',
 'Location_Broomfield, CO',
 'Location_Burbank, CA',
 'Location_Burlingame, CA',
 'Location_Burlingame, KS',
 'Location_Camden, NJ',
 'Location_Campbell, CA',
 'Location_Carmel, IN',
 'Location_C

In [22]:
# You need to reset the column order for the colunmn elements making up the "Size", "Type of Ownership",
# and the "Revenue" columns. Otherwise the "Size", "Type of Ownership" and "Revenue" appear in some type of 
# non-logical order.
cols2 = ['Rating',
 'Location_Addison, TX',
 'Location_Alachua, FL',
 'Location_Alameda, CA',
 'Location_Alhambra, CA',
 'Location_Allegheny West, PA',
 'Location_Allen, TX',
 'Location_American Fork, UT',
 'Location_Anaheim, CA',
 'Location_Arcadia, CA',
 'Location_Arlington Heights, IL',
 'Location_Arlington, TX',
 'Location_Athens, GA',
 'Location_Aurora, CO',
 'Location_Austin, TX',
 'Location_Azusa, CA',
 'Location_Beech Grove, IN',
 'Location_Bellevue, WA',
 'Location_Bensalem, PA',
 'Location_Berkeley Heights, NJ',
 'Location_Berkeley, CA',
 'Location_Berwyn, PA',
 'Location_Beverly Hills, CA',
 'Location_Blue Bell, PA',
 'Location_Boothwyn, PA',
 'Location_Boulder, CO',
 'Location_Brea, CA',
 'Location_Bridgeview, IL',
 'Location_Broadview, IL',
 'Location_Bronx, NY',
 'Location_Brooklyn, NY',
 'Location_Broomfield, CO',
 'Location_Burbank, CA',
 'Location_Burlingame, CA',
 'Location_Burlingame, KS',
 'Location_Camden, NJ',
 'Location_Campbell, CA',
 'Location_Carmel, IN',
 'Location_Carrollton, TX',
 'Location_Cedar Park, TX',
 'Location_Centennial, CO',
 'Location_Cerritos, CA',
 'Location_Chandler, AZ',
 'Location_Charlotte, NC',
 'Location_Chesapeake, VA',
 'Location_Chester Township, PA',
 'Location_Chicago, IL',
 'Location_City of Industry, CA',
 'Location_Columbus, OH',
 'Location_Conshohocken, PA',
 'Location_Coppell, TX',
 'Location_Culver City, CA',
 'Location_Cupertino, CA',
 'Location_DC Ranch, AZ',
 'Location_Dallas, TX',
 'Location_Daly City, CA',
 'Location_Deerfield, IL',
 'Location_Denver, CO',
 'Location_Downers Grove, IL',
 'Location_Doylestown, PA',
 'Location_Draper, UT',
 'Location_Dublin, OH',
 'Location_East Palo Alto, CA',
 'Location_El Cajon, CA',
 'Location_El Segundo, CA',
 'Location_Elk Grove Village, IL',
 'Location_Emeryville, CA',
 'Location_Englewood, CO',
 'Location_Essex Fells, NJ',
 'Location_Evanston, IL',
 'Location_Exton, PA',
 'Location_Farmers Branch, TX',
 'Location_Feasterville Trevose, PA',
 'Location_Florham Park, NJ',
 'Location_Fort Eustis, VA',
 'Location_Fort Lee, NJ',
 'Location_Fort Mill, SC',
 'Location_Fort Washington, PA',
 'Location_Fort Worth, TX',
 'Location_Foster City, CA',
 'Location_Franklin Lakes, NJ',
 'Location_Fremont, CA',
 'Location_Gainesville, FL',
 'Location_Gardena, CA',
 'Location_Glendale, AZ',
 'Location_Glendale, CA',
 'Location_Glenview, IL',
 'Location_Grapevine, TX',
 'Location_Greenwood Village, Arapahoe, CO',
 'Location_Hampton, VA',
 'Location_Hanford, CA',
 'Location_Harrison, NY',
 'Location_Haworth, NJ',
 'Location_Henderson, CO',
 'Location_Hercules, CA',
 'Location_Hilliard, OH',
 'Location_Hoboken, NJ',
 'Location_Horsham, PA',
 'Location_Houston, TX',
 'Location_Huntersville, NC',
 'Location_Indianapolis, IN',
 'Location_Inglewood, CA',
 'Location_Irving, TX',
 'Location_Irwindale, CA',
 'Location_Iselin, NJ',
 'Location_Issaquah, WA',
 'Location_Itasca, IL',
 'Location_Jacksonville, FL',
 'Location_Jeffersonville, IN',
 'Location_Jersey City, NJ',
 'Location_Kent, WA',
 'Location_King of Prussia, PA',
 'Location_Kirkland, WA',
 'Location_Lackland AFB, TX',
 'Location_Lake Success, NY',
 'Location_Lakewood, CO',
 'Location_Lawrence, IN',
 'Location_Lehi, UT',
 'Location_Lewisville, TX',
 'Location_Little Ferry, NJ',
 'Location_Littleton, CO',
 'Location_Lone Tree, CO',
 'Location_Long Beach, CA',
 'Location_Long Island City, NY',
 'Location_Los Angeles, CA',
 'Location_Los Gatos, CA',
 'Location_Louisville, CO',
 'Location_Malvern, PA',
 'Location_Manhasset, NY',
 'Location_Marin City, CA',
 'Location_Marina del Rey, CA',
 'Location_Marlton, NJ',
 'Location_Maywood, IL',
 'Location_Menlo Park, CA',
 'Location_Mesa, AZ',
 'Location_Millbrae, CA',
 'Location_Milpitas, CA',
 'Location_Montvale, NJ',
 'Location_Moorestown, NJ',
 'Location_Mooresville, NC',
 'Location_Mount Laurel, NJ',
 'Location_Mount Vernon, NY',
 'Location_Mountain View, CA',
 'Location_New York, NY',
 'Location_Newark, CA',
 'Location_Newark, NJ',
 'Location_Newport News, VA',
 'Location_Newtown Square, PA',
 'Location_Norfolk, VA',
 'Location_Northbrook, IL',
 'Location_Northfield, IL',
 'Location_Northlake, IL',
 'Location_Northridge, CA',
 'Location_Novato, CA',
 'Location_Oak Brook, IL',
 'Location_Oakland, CA',
 'Location_Palo Alto, CA',
 'Location_Paoli, PA',
 'Location_Parsippany, NJ',
 'Location_Pasadena, CA',
 'Location_Pasadena, TX',
 'Location_Pearland, TX',
 'Location_Philadelphia, PA',
 'Location_Phoenix, AZ',
 'Location_Pico Rivera, CA',
 'Location_Plano, TX',
 'Location_Pleasanton, CA',
 'Location_Plymouth Meeting, PA',
 'Location_Portsmouth, VA',
 'Location_Queens Village, NY',
 'Location_Radnor, PA',
 'Location_Redmond, WA',
 'Location_Redwood City, CA',
 'Location_Reedley, CA',
 'Location_Renton, WA',
 'Location_Richardson, TX',
 'Location_Ridley Park, PA',
 'Location_Riverton, UT',
 'Location_Rolling Meadows, IL',
 'Location_Round Rock, TX',
 'Location_Salt Lake City, UT',
 'Location_San Antonio, TX',
 'Location_San Diego, CA',
 'Location_San Francisco, CA',
 'Location_San Jose, CA',
 'Location_San Mateo, CA',
 'Location_San Rafael, CA',
 'Location_San Ramon, CA',
 'Location_Santa Clara, CA',
 'Location_Santa Monica, CA',
 'Location_Scottsdale, AZ',
 'Location_Seattle, WA',
 'Location_Secaucus, NJ',
 'Location_Smithfield, VA',
 'Location_South Plainfield, NJ',
 'Location_South San Francisco, CA',
 'Location_Southlake, TX',
 'Location_Spring, TX',
 'Location_Stafford, TX',
 'Location_Stanford, CA',
 'Location_Staten Island, NY',
 'Location_Suffolk, VA',
 'Location_Sugar Land, TX',
 'Location_Summit, NJ',
 'Location_Sunnyvale, CA',
 'Location_Tarrant, TX',
 'Location_Tempe, AZ',
 'Location_Topeka, KS',
 'Location_Torrance, CA',
 'Location_Union City, CA',
 'Location_Venice, CA',
 'Location_Virginia Beach, VA',
 'Location_Visalia, CA',
 'Location_Walnut Creek, CA',
 'Location_Wayne, PA',
 'Location_Weehawken, NJ',
 'Location_West Chester, PA',
 'Location_West Conshohocken, PA',
 'Location_West Jordan, UT',
 'Location_West Orange, NJ',
 'Location_West Point, PA',
 'Location_Westerville, OH',
 'Location_Westlake, TX',
 'Location_Whippany, NJ',
 'Location_Whitestown, IN',
 'Location_Whittier, CA',
 'Location_Wilmington, DE',
 'Location_Woodbridge, NJ',
 'Location_Woodcliff Lake, NJ',
 'Location_Woodland Hills, CA',
 'Location_Woodridge, IL',
 'Location_Yorktown, VA',
 'Size_Unknown',
 'Size_1 to 50 employees',
 'Size_51 to 200 employees',
 'Size_201 to 500 employees',
 'Size_501 to 1000 employees',
 'Size_1001 to 5000 employees',        
 'Size_5001 to 10000 employees',
 'Size_10000+ employees',
 'Type of ownership_Unknown',
 'Type of ownership_College / University',
 'Type of ownership_Company - Private',
 'Type of ownership_Company - Public',
 'Type of ownership_Contract',
 'Type of ownership_Franchise',
 'Type of ownership_Government',
 'Type of ownership_Hospital',
 'Type of ownership_Nonprofit Organization',
 'Type of ownership_Other Organization',
 'Type of ownership_Private Practice / Firm',
 'Type of ownership_School / School District',
 'Type of ownership_Self-employed',
 'Type of ownership_Subsidiary or Business Segment',
 'Industry_Accounting',
 'Industry_Advertising & Marketing',
 'Industry_Aerospace & Defense',
 'Industry_Architectural & Engineering Services',
 'Industry_Audiovisual',
 'Industry_Automotive Parts & Accessories Stores',
 'Industry_Banks & Credit Unions',
 'Industry_Beauty & Personal Accessories Stores',
 'Industry_Biotech & Pharmaceuticals',
 'Industry_Brokerage Services',
 'Industry_Building & Personnel Services',
 'Industry_Cable, Internet & Telephone Providers',
 'Industry_Casual Restaurants',
 'Industry_Catering & Food Service Contractors',
 'Industry_Chemical Manufacturing',
 'Industry_Colleges & Universities',
 'Industry_Commercial Equipment Repair & Maintenance',
 'Industry_Computer Hardware & Software',
 'Industry_Construction',
 'Industry_Consulting',
 'Industry_Consumer Electronics & Appliances Stores',
 'Industry_Consumer Product Rental',
 'Industry_Consumer Products Manufacturing',
 'Industry_Convenience Stores & Truck Stops',
 'Industry_Department, Clothing, & Shoe Stores',
 'Industry_Drug & Health Stores',
 'Industry_Education Training Services',
 'Industry_Electrical & Electronic Manufacturing',
 'Industry_Energy',
 'Industry_Enterprise Software & Network Solutions',
 'Industry_Express Delivery Services',
 'Industry_Federal Agencies',
 'Industry_Financial Analytics & Research',
 'Industry_Financial Transaction Processing',
 'Industry_Food & Beverage Manufacturing',
 'Industry_Food & Beverage Stores',
 'Industry_Gambling',
 'Industry_Gas Stations',
 'Industry_General Merchandise & Superstores',
 'Industry_General Repair & Maintenance',
 'Industry_Grocery Stores & Supermarkets',
 'Industry_Health Care Products Manufacturing',
 'Industry_Health Care Services & Hospitals',
 'Industry_Health Fundraising Organizations',
 'Industry_Health, Beauty, & Fitness',
 'Industry_Home Centers & Hardware Stores',
 'Industry_Hotels, Motels, & Resorts',
 'Industry_IT Services',
 'Industry_Industrial Manufacturing',
 'Industry_Insurance Agencies & Brokerages',
 'Industry_Insurance Carriers',
 'Industry_Internet',
 'Industry_Investment Banking & Asset Management',
 'Industry_K-12 Education',
 'Industry_Legal',
 'Industry_Lending',
 'Industry_Logistics & Supply Chain',
 'Industry_Membership Organizations',
 'Industry_Metals Brokers',
 'Industry_Miscellaneous Manufacturing',
 'Industry_Motion Picture Production & Distribution',
 'Industry_Municipal Governments',
 'Industry_News Outlet',
 'Industry_Oil & Gas Services',
 'Industry_Other Retail Stores',
 'Industry_Pet & Pet Supplies Stores',
 'Industry_Preschool & Child Care',
 'Industry_Publishing',
 'Industry_Real Estate',
 'Industry_Research & Development',
 'Industry_Security Services',
 'Industry_Social Assistance',
 'Industry_Sporting Goods Stores',
 'Industry_Sports & Recreation',
 'Industry_Staffing & Outsourcing',
 'Industry_State & Regional Agencies',
 'Industry_Stock Exchanges',
 'Industry_TV Broadcast & Cable Networks',
 'Industry_Telecommunications Services',
 'Industry_Transportation Equipment Manufacturing',
 'Industry_Transportation Management',
 'Industry_Truck Rental & Leasing',
 'Industry_Trucking',
 'Industry_Utilities',
 'Industry_Vehicle Dealers',
 'Industry_Venture Capital & Private Equity',
 'Industry_Video Games',
 'Industry_Wholesale',
 'Sector_Accounting & Legal',
 'Sector_Aerospace & Defense',
 'Sector_Arts, Entertainment & Recreation',
 'Sector_Biotech & Pharmaceuticals',
 'Sector_Business Services',
 'Sector_Construction, Repair & Maintenance',
 'Sector_Consumer Services',
 'Sector_Education',
 'Sector_Finance',
 'Sector_Government',
 'Sector_Health Care',
 'Sector_Information Technology',
 'Sector_Insurance',
 'Sector_Manufacturing',
 'Sector_Media',
 'Sector_Mining & Metals',
 'Sector_Non-Profit',
 'Sector_Oil, Gas, Energy & Utilities',
 'Sector_Real Estate',
 'Sector_Restaurants, Bars & Food Services',
 'Sector_Retail',
 'Sector_Telecommunications',
 'Sector_Transportation & Logistics',
 'Sector_Travel & Tourism',
 'Revenue_Unknown / Non-Applicable',
 'Revenue_Less than $1 million (USD)',
 'Revenue_$1 to $5 million (USD)',
 'Revenue_$5 to $10 million (USD)',
 'Revenue_$10 to $25 million (USD)',
 'Revenue_$25 to $50 million (USD)',
 'Revenue_$50 to $100 million (USD)',
 'Revenue_$100 to $500 million (USD)',
 'Revenue_$500 million to $1 billion (USD)',
 'Revenue_$1 to $2 billion (USD)',
 'Revenue_$2 to $5 billion (USD)',
 'Revenue_$5 to $10 billion (USD)',
 'Revenue_$10+ billion (USD)']

In [23]:
# Reset the order of the columns
X_dummies = X_dummies[cols2]
X_dummies

,Rating,"Location_Addison, TX","Location_Alachua, FL","Location_Alameda, CA","Location_Alhambra, CA","Location_Allegheny West, PA","Location_Allen, TX","Location_American Fork, UT","Location_Anaheim, CA","Location_Arcadia, CA",...,Revenue_$5 to $10 million (USD),Revenue_$10 to $25 million (USD),Revenue_$25 to $50 million (USD),Revenue_$50 to $100 million (USD),Revenue_$100 to $500 million (USD),Revenue_$500 million to $1 billion (USD),Revenue_$1 to $2 billion (USD),Revenue_$2 to $5 billion (USD),Revenue_$5 to $10 billion (USD),Revenue_$10+ billion (USD)
0,3.2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,3.8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3.4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4.1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,3.9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1851,4.1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1852,2.5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1853,2.9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1854,3.1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [24]:
X_dummies.keys()

Index(['Rating', 'Location_Addison, TX', 'Location_Alachua, FL',
       'Location_Alameda, CA', 'Location_Alhambra, CA',
       'Location_Allegheny West, PA', 'Location_Allen, TX',
       'Location_American Fork, UT', 'Location_Anaheim, CA',
       'Location_Arcadia, CA',
       ...
       'Revenue_$5 to $10 million (USD)', 'Revenue_$10 to $25 million (USD)',
       'Revenue_$25 to $50 million (USD)', 'Revenue_$50 to $100 million (USD)',
       'Revenue_$100 to $500 million (USD)',
       'Revenue_$500 million to $1 billion (USD)',
       'Revenue_$1 to $2 billion (USD)', 'Revenue_$2 to $5 billion (USD)',
       'Revenue_$5 to $10 billion (USD)', 'Revenue_$10+ billion (USD)'],
      dtype='object', length=379)

In [25]:
y_label = LabelEncoder().fit_transform(df['NewSalaryEstimate'])
y_label

array([1, 1, 1, ..., 5, 5, 5])

In [26]:
df["NewSalaryEstimate"].unique()

array(['$20K-$50K', '$56K-$70K', '$71K-$80K', '$81K-$90K', '$91K-$109K',
       '$110K-$190K'], dtype=object)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_label, random_state=1)

In [28]:
# Scale the X data using StandardScaler()
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_train_scaled

array([[ 1.02564092, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259, -0.33199946],
       [ 0.25668683, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259, -0.33199946],
       [-0.66605807, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259,  3.01205308],
       ...,
       [ 0.10289601, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259, -0.33199946],
       [-0.51226726, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259,  3.01205308],
       [-0.0508948 , -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259, -0.33199946]])

In [29]:
# Transform the test dataset based on the fit from the training data
X_test_scaled = scaler.transform(X_test)
X_test_scaled

array([[ 1.94838582, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259, -0.33199946],
       [ 1.94838582, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259, -0.33199946],
       [ 0.10289601, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259, -0.33199946],
       ...,
       [ 0.10289601, -0.02683174, -0.02683174, ..., -0.28585322,
         5.80947502, -0.33199946],
       [ 0.10289601, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259, -0.33199946],
       [-0.51226726, -0.02683174, -0.02683174, ..., -0.28585322,
        -0.17213259,  3.01205308]])

In [30]:
#X_train_scaled.to_csv("temp_scaled.csv")
#np.savetxt("temp_scaled.csv", X_train_scaled, delimiter=",")

In [31]:
#np.savetxt("tempy_scaled.csv", y_train, delimiter=",")

In [32]:
# Train a Random Forest Classifier model on the scaled data and print the model score
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

Training Score: 0.8964028776978418
Testing Score: 0.38146551724137934


In [33]:
#Make predictions
predictions = clf.predict(X_test_scaled)
print(f"First 10 Predictions:   {predictions[:10]}")
print(f"First 10 Actual labels: {y_test[:10].tolist()}")

First 10 Predictions:   [3 5 1 3 2 4 2 1 2 2]
First 10 Actual labels: [3, 5, 5, 4, 2, 2, 1, 3, 3, 1]


In [34]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,3,3
1,5,5
2,1,5
3,3,4
4,2,2
...,...,...
459,1,0
460,2,2
461,3,1
462,5,5


In [35]:
# Save
joblib.dump(clf, "ML_models.sav")

['ML_models.sav']